<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

# Imports

In [2]:
import numpy as np
import gensim
import os
import re
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore
import pandas as pd
import spacy
#import pyLDAvis.gensim

In [3]:
nlp = spacy.load("en_core_web_lg")

# Load Data

In [4]:
df = pd.read_csv("../module1-text-data/data/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv")

In [5]:
df.head(1)

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.didPurchase,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVpgNzjwLJeJML43Kpxn,2015-10-30T08:59:32Z,2019-04-25T09:08:16Z,AmazonBasics AAA Performance Alkaline Batterie...,"B00QWO9P0O,B00LH3DMUO",Amazonbasics,"AA,AAA,Health,Electronics,Health & Household,C...",Health & Beauty,https://images-na.ssl-images-amazon.com/images...,"amazonbasics/hl002619,amazonbasicsaaaperforman...",...,NaN,NaN,NaN,NaN,3,https://www.amazon.com/product-reviews/B00QWO9...,I order 3 of them and one of the item is bad q...,... 3 of them and one of the item is bad quali...,Byger yang,"https://www.barcodable.com/upc/841710106442,ht..."


# Clean

In [6]:
df.drop(['id', 'dateAdded', 'dateUpdated', 'asins',
         'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer',
         'manufacturerNumber', 'reviews.date', 'reviews.dateSeen',
         'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id',
         'reviews.numHelpful', 'reviews.sourceURLs', 'reviews.title',
        'reviews.username', 'sourceURLs'], axis=1, inplace=True)
df.head(1)

,name,brand,reviews.rating,reviews.text
0,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,3,I order 3 of them and one of the item is bad q...


In [7]:
df['reviews.text'] = df['reviews.text'].str.lower()
df['reviews.text'] = df['reviews.text'].str.split()
df.head(1)

,name,brand,reviews.rating,reviews.text
0,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,3,"[i, order, 3, of, them, and, one, of, the, ite..."


In [30]:
df.name.nunique()

/home/allan/.local/share/virtualenvs/u4-s1-1fz81KSO/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


65

# Stop word

In [10]:
stop_words = [
    "amazon",
    "review"
]
stops = nlp.Defaults.stop_words.union(stop_words)

In [11]:
def remove_stops(_list_):
    keywords = []
    for keyword in _list_:
        phrase = []
        words = keyword.split()
        for word in words:
            if word not in stops:
                phrase.append(word)
        phrase = ' '.join(phrase)
        if len(phrase) > 0:
            keywords.append(phrase)
    return keywords

In [12]:
df['keywords'] = df['reviews.text'].apply(remove_stops)

In [13]:
df.head(1)

,name,brand,reviews.rating,reviews.text,keywords
0,AmazonBasics AAA Performance Alkaline Batterie...,Amazonbasics,3,"[i, order, 3, of, them, and, one, of, the, ite...","[order, 3, item, bad, quality., missing, backu..."


# Gensim

In [14]:
id2word = corpora.Dictionary(df.keywords)
id2word.filter_extremes(no_below=.05, no_above=.95)
corpus = [id2word.doc2bow(text) for text in df.keywords]

In [23]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   num_topics=7,
                   passes=50,
                   workers=8)

/home/allan/.local/share/virtualenvs/u4-s1-1fz81KSO/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
words = [re.findall(r'"([^"]*)"', t[1]) for t in lda.print_topics(65)]
topics = [", ".join(t[:5]) for t in words]

/home/allan/.local/share/virtualenvs/u4-s1-1fz81KSO/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

/home/allan/.local/share/virtualenvs/u4-s1-1fz81KSO/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

/home/allan/.local/share/virtualenvs/u4-s1-1fz81KSO/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.267216 -0.078636       1        1  13.147501
3      0.216461 -0.017206       2        1  10.218836
6      0.033749  0.063928       3        1   7.265104
0     -0.146457  0.148282       4        1   6.872034
7     -0.205534 -0.275748       5        1   6.590440
16     0.076722 -0.168886       6        1   6.448830
12     0.035013  0.005914       7        1   6.423895
9      0.149492 -0.089158       8        1   5.992019
15     0.109018 -0.125146       9        1   5.865964
11     0.107021  0.043365      10        1   5.558825
10     0.013714  0.027645      11        1   4.230414
13     0.084376  0.056889      12        1   3.957205
1      0.006918  0.079133      13        1   3.775560
14     0.046924  0.042440      14        1   3.735243
5     -0.120048  0.086701      15        1   3.669857
8     -0.120461  0.125885      16        1   3.488164
2     -0.019693  0.074599      17        1   2.760109, topic_info=           Term         Freq        Total Category  logprob  loglift
66         good  4636.000000  4636.000000  Default  30.0000  30.0000
20    batteries  5065.000000  5065.000000  Default  29.0000  29.0000
77        great  7485.000000  7485.000000  Default  28.0000  28.0000
4480      loves  2066.000000  2066.000000  Default  27.0000  27.0000
26         long  2006.000000  2006.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
7810     tablet    40.034428  4671.959157  Topic17  -5.4910  -1.1697
724     replace    29.807839   332.559991  Topic17  -5.7860   1.1779
73        can't    31.736632   850.795791  Topic17  -5.7233   0.3012
1032    quality    31.621723   923.742369  Topic17  -5.7269   0.2153
14         like    32.722338  2290.195148  Topic17  -5.6927  -0.6584

[1353 rows x 6 columns], token_table=       Topic      Freq Term
term                       
2213       2  0.060323    !
2213       4  0.086176    !
2213       5  0.620465    !
2213       8  0.025853    !
2213      10  0.077558    !
...      ...       ...  ...
11807      2  0.745159   yr
11807      9  0.019355   yr
11807     12  0.053226   yr
11807     17  0.179032   yr
15182      2  0.915116  yr.

[4241 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 7, 1, 8, 17, 13, 10, 16, 12, 11, 14, 2, 15, 6, 9, 3])

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling

# Class stuff

In [1]:
# import numpy as np
# import gensim
# import os
# import re
# from gensim.utils import simple_preprocess
# from gensim.parsing.preprocessing import STOPWORDS
# from gensim import corpora
# from gensim.models.ldamulticore import LdaMulticore
# import pandas as pd
# import tqdm
# from ast import literal_eval
# import spacy
# #import pyLDAvis.gensim
# # from tqdm import tqdm, using tqdm.pandas() to see preogress bar

# Read in data

In [2]:
# df = pd.read_csv("data/imbd_keywords.csv")
# df.head()

In [3]:
# df.keywords = df.keywords.apply(literal_eval)

In [4]:
# df.head()

In [5]:
# stop_words = [
#     "movie",
#     "film",
#     "review",
#     "actor",
#     "actress"
# ]
# nlp = spacy.load("en_core_web_lg")

In [6]:
# stops = nlp.Defaults.stop_words.union(stop_words)

In [7]:
# def remove_stop(ls):
#     x = []
#     for words in ls:
#         temp = words.split()
#         for word in temp:
#             if word in stops:
#                 temp.remove(word)
#             x.append(" ".join(temp))
#     return x
# # there was more but i didn't like his function...

In [8]:
# df.keywords = df.keywords.apply(remove_stop)
# #df.keywords[0]

In [9]:
# import gensim
# from gensim import corpora
# from gensim.models.ldamulticore import LdaMulticore

In [10]:
# id2word = corpora.Dictionary(df.keywords)

In [11]:
# id2word.token2id(['sea-monster'])

In [12]:
# len(id2word.keys())

In [13]:
# id2word.filter_extremes(no_below=5, no_above=.95)

In [14]:
# len(id2word.keys())

In [15]:
# corpus = [id2word.doc2bow(text) for text in df.keywords]

In [16]:
# corpus[812]

In [17]:
# id2word[86]

In [18]:
# lda = LdaMulticore(corpus=corpus,
#                    id2word=id2word,
#                    num_topics=50,
#                    passes=50,
#                    workers=8)

In [19]:
# lda.print_topics()[0]  # id, words

In [20]:
# import re

# words = [re.findall(r'"([^"]*)"', t[1]) for t in lda.print_topics(50)]

In [21]:
# words[0]

In [22]:
# topics = [", ".join(t[:5]) for t in words]A

In [23]:
# topics

In [24]:
# import pyLDAvis.gensim

# pyLDAvis.enable_notebook()

In [25]:
#pyLDAvis.gensim.prepare(lda, corpus, id2word)

In [26]:
#disto = [lda[doc] for doc in corpus]

In [27]:
# def update(doc):
#     d_dist = {k:0 for k in range(21)}
#     for t in doc:
#         d_dist[t[0]] = t[1]
#     return d_dist

# new_distro = [update(doc) for doc in disto]

In [28]:
# topic_df = pd.DataFrame.from_records(new_distro)
# topic_df.columns = topics

In [29]:
# topic_df.head(1)